In [41]:
# imports
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np

# settings view dataframes
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)


# find dataset
citibike_data = [
    Path('data') / 'citibike-tripdata.csv',
    Path('..') / 'data' / 'citibike-tripdata.csv',
    Path('../..') / 'data' / 'citibike-tripdata.csv',
]
for p in citibike_data:
    if p.is_file():
        DATA_PATH = p
        break
else:
    raise FileNotFoundError('Не найден citibike-tripdata.csv')

print('Использую файл:', DATA_PATH.resolve())

# load dataset
citibike_data = pd.read_csv(DATA_PATH)

# fast sanity-check
print('Форма набора данных (кол-во строк и столбцов):', citibike_data.shape)
print('Первые 23 столбца:', list(citibike_data.columns[:23]))

Использую файл: /Users/kirilltishchenko/lessonsPandas/data/citibike-tripdata.csv
Форма набора данных (кол-во строк и столбцов): (300000, 14)
Первые 23 столбца: ['starttime', 'stoptime', 'start station id', 'start station name', 'start station latitude', 'start station longitude', 'end station id', 'end station name', 'end station latitude', 'end station longitude', 'bikeid', 'usertype', 'birth year', 'gender']


In [3]:
# Сколько пропусков в столбце start station id?
missing_start_station_id = citibike_data['start station id'].isna().sum()
missing_start_station_id

np.int64(169)

In [5]:
# Какой тип данных имеют столбцы starttime и stoptime?
citibike_data[['starttime', 'stoptime']].dtypes

starttime    object
stoptime     object
dtype: object

In [6]:
# Найдите идентификатор самой популярной стартовой стоянки. Запишите идентификатор в виде целого числа.
# Частота встречаемости можно найти с помощью метода mode().
popular_start_station_id = citibike_data['start station id'].mode()[0]
popular_start_station_id

np.float64(281.0)

In [7]:
# Велосипед с каким идентификатором является самым популярным?
popular_bike_id = citibike_data['bikeid'].mode()[0]
popular_bike_id

np.int64(33887)

In [16]:
#Строит Series с долями (не количеством) по каждому значению столбца usertype.
# normalize=True — вернуть частоты в долях, сумма ≈ 1.
# dropna=True — пропуски игнорируются.
# Индекс — сами типы (например, Subscriber, Customer), значения — их доли в диапазоне [0, 1].
vc = citibike_data['usertype'].value_counts(normalize=True, dropna=True)

# Возвращает метку индекса (тип пользователя) с максимальной долей.
# Если несколько типов с одинаковым максимумом, вернётся первый по порядку.
dominant_type = vc.idxmax()

# Числовое значение максимальной доли (float от 0 до 1) для преобладающего типа.
share = vc.max()  # доля в диапазоне 0..1

print(f'Преобладающий тип: {dominant_type}; доля: {share:.3f}')
share

Преобладающий тип: Subscriber; доля: 0.774


np.float64(0.7740066666666666)

In [35]:
# Кто больше занимается велоспортом — мужчины или женщины? 
# В ответ запишите число поездок для той группы, у которой их больше.
# В колонке 'gender' след. знач: 0 — неизвестный, 1 — мужчина, 2 — женщина
gen = citibike_data['gender'].value_counts(normalize=True, dropna=True)
dom_max_gen = gen.idxmax()
if dom_max_gen == 1:
    print("Больше занимается велоспортом — Мужчины")
elif dom_max_gen == 2:
    print("больше занимается велоспортом — Женщины")
    
count_men_max = citibike_data['gender'].value_counts()[dom_max_gen]
print(f'Кол-во поездок у мужчин {count_men_max}')

# В данной ячейке мое решение, а в следующей ячейки решение GPT-5 
# У GPT-5 не верное решение и длинное


Больше занимается велоспортом — Мужчины
Кол-во поездок у мужчин 183582


In [36]:
# Сгенерировал ИИ (GPT-5)
# Кто больше совершает поездок — мужчины или женщины? Вывести большее число поездок
# Ищем подходящее имя колонки пола среди распространённых в наборах CitiBike
possible_gender_cols = [
    'gender', 'Gender', 'sex', 'Sex',
    'member_gender', 'Member Gender',
    'rider_gender', 'Rider Gender',
]

# Находим первую существующую колонку
for col in possible_gender_cols:
    if col in citibike_data.columns:
        gender_col = col
        break
else:
    # Иногда пол кодируется 0/1/2 в столбце 'gender' с описанием в документации,
    # но если такой колонки нет, аккуратно сообщим, какие есть текстовые кандидаты
    text_cols = [c for c, dt in citibike_data.dtypes.items() if dt == 'object']
    raise KeyError(f"Не найден столбец пола. Проверьте колонки: {citibike_data.columns.tolist()}\n"
                   f"Текстовые колонки для проверки: {text_cols}")

# Приводим значения к унифицированному виду
s = citibike_data[gender_col]
# Часто встречается кодировка: 0 - unknown, 1 - male, 2 - female
if s.dropna().apply(lambda x: str(x).strip()).isin({'0','1','2'}).all():
    mapping = {0: 'male', 1: 'male', 2: 'female', '0': 'unknown', '1': 'male', '2': 'female'}
    s_norm = s.map(mapping).fillna('unknown')
else:
    s_norm = s.astype(str).str.strip().str.lower().replace({'m': 'male', 'f': 'female'})

counts = s_norm.value_counts(dropna=False)
men = int(counts.get('male', 0))
women = int(counts.get('female', 0))

result = max(men, women)
print(f"Мужчины: {men}, Женщины: {women}. Большее число поездок: {result}")
result




Мужчины: 225494, Женщины: 74506. Большее число поездок: 225494


225494

In [38]:
# Удаляем идентификаторы стартовой и конечной стоянок и считаем оставшиеся столбцы

# Число столбцов в таблице до удаления (в данном блоке хранится для 
# контроля, дальше не используется).
n_before = citibike_data.shape[1]

# Имена колонок, которые нужно удалить (в нижнем регистре для удобства сравнения).
id_aliases = {
    'start station id', 'end station id',
    'start_station_id', 'end_station_id'
}

# Имена колонок, которые нужно удалить (в исходном регистре).
# Проходим по всем колонкам и выбираем те, которые есть в множестве id_aliases.
cols_to_drop = [c for c in citibike_data.columns if c.lower() in id_aliases]

# Удаляем указанные колонки из DataFrame.
citibike_data = citibike_data.drop(columns=cols_to_drop, errors='ignore')

# Число столбцов в таблице после удаления.
n_after = citibike_data.shape[1]
print('Удалены колонки:', cols_to_drop)
print('Осталось столбцов:', n_after)
n_after 

Удалены колонки: []
Осталось столбцов: 12


12

In [39]:
citibike_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   starttime                300000 non-null  object 
 1   stoptime                 300000 non-null  object 
 2   start station name       299831 non-null  object 
 3   start station latitude   300000 non-null  float64
 4   start station longitude  300000 non-null  float64
 5   end station name         299831 non-null  object 
 6   end station latitude     300000 non-null  float64
 7   end station longitude    300000 non-null  float64
 8   bikeid                   300000 non-null  int64  
 9   usertype                 300000 non-null  object 
 10  birth year               300000 non-null  int64  
 11  gender                   300000 non-null  int64  
dtypes: float64(4), int64(3), object(5)
memory usage: 27.5+ MB


In [ ]:
#
REF_YEAR = 2018
# новый столбец age; тип Int64 — целые с поддержкой <NA> для пропусков
#citibike_data['age'] = (REF_YEAR - citibike_data['birth year']).astype('Int64')

# удаляем исходный столбец с годом рождения
#citibike_data = citibike_data.drop(columns='birth year')
# быстрая проверка
#citibike_data[['age']].head()
# число поездок (строк) с возрастом > 60
count_60plus = citibike_data.loc[citibike_data['age'] > 60].shape[0]

count_60plus


11837

In [ ]:
# Преобразуем столбцы starttime и stoptime в тип datetime
citibike_data['starttime'] = pd.to_datetime(citibike_data['starttime'], errors='coerce')
citibike_data['stoptime']  = pd.to_datetime(citibike_data['stoptime'],  errors='coerce')

# Новый столбец с длительностью поездки
citibike_data['trip_duration'] = citibike_data['stoptime'] - citibike_data['starttime']
citibike_data['trip_duration']

# Вычислите целое число минут для поездки с индексом 3.
trip_minutes = (citibike_data['trip_duration'].dt.total_seconds() // 60).astype('Int64')

# Целое число минут для поездки с индексом 3
minutes_idx3 = trip_minutes.loc[3]

print('Целых минут для поездки с индексом 3:', minutes_idx3)
minutes_idx3

Целых минут для поездки с индексом 3: 7


np.int64(7)

In [64]:
# Гарантируем тип datetime
citibike_data['starttime'] = pd.to_datetime(citibike_data['starttime'], errors='coerce')

# Признак-мигалка: 1 для сб/вс, иначе 0
citibike_data['weekend'] = citibike_data['starttime'].dt.dayofweek.isin([5, 6]).astype('Int8')

# Сколько поездок началось в выходные
weekend_count = int(citibike_data['weekend'].sum())
print('Поездок, начавшихся в выходные:', weekend_count)
weekend_count  # ожидаемый ответ: 115135



Поездок, начавшихся в выходные: 115135


115135

In [ ]:
# Гарантируем datetime
citibike_data['starttime'] = pd.to_datetime(citibike_data['starttime'], errors='coerce')

# Час начала
h = citibike_data['starttime'].dt.hour

# Категории времени суток:
# night:   0..6  (включительно)
# morning: 6..12 (12 включ., 6 исключ.)
# day:     12..18
# evening: 18..23
citibike_data['time_of_day'] = pd.cut(   # разбивает числовую Series h на интервалы и возвращает категориальные метки.
    h, bins=[-0.1, 6, 12, 18, 23],       # задаёт границы интервалов
    labels=['night', 'morning', 'day', 'evening'],  # присваивает названия интервалам: night, morning, day, evening
    right=True, include_lowest=True     # при right=True интервалы правозамкнутые: [-0.1,6], (6,12], (12,18], (18,23]. include_lowest=True включает нижнюю границу первого интервала (нужно, чтобы 0 попал в night; -0.1 — технический трюк).
)

# Во сколько раз «day» больше «night» (округление до целого)
counts = citibike_data['time_of_day'].value_counts()
ratio_rounded = int(round(counts.get('day', 0) / counts.get('night', 1)))
print('Во сколько раз day > night:', ratio_rounded)
ratio_rounded

Во сколько раз day > night: 9


9